## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. They often do that by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. And it is reactive, the configuration changes are already in the network. 

Batfish allows network engineers to proactively validate BGP configuration to ensure sessions are compatibly configured and will be established, thereby avoiding potential network outages.

In this notebook, we will look at how you can extract BGP configuration and session status information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

load_questions()

### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_n"
SNAPSHOT_NAME = "example_s"

SNAPSHOT_PATH = "networks/basic/networks/fattree6"
# SNAPSHOT_PATH = "networks/example-bgp"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_s'

In [3]:
# help(bf_upload_diagnostics)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Batfish BGP related questions


Batfish has a number of questions dedicated to extracting BGP configuration and state information. This notebook will show you how to use each of them to extract information you need to validate a configuration change or troubleshoot a BGP issue

1) `bfq.bgpProcessConfiguration` - This will return common information about the BGP process: router id, whether the device is acting as a route-reflector, multipath settings, list of neighbors, etc.

2) `bfq.bgpPeerConfiguration` - This will return the list of neighbors for the given node(s) and specific attributes for each configured neighbor such as: device local IP address, Local Interface (only use for BGP unnumbered sessions), remote-as, remote/peer/neighbor IP, peer-group, import and export policy, etc.

3) `bfq.bgpSessionCompatbility` - For each configured BGP peer, this question will return the `Configured_Status`. This is Batfish's view of whether or not a given BGP session is properly configured. Any peer that does not have `Configured_Status` equal to `UNIQUE_MATCH` or `DYNAMIC_MATCH` is misconfigured.

4) `bfq.bgpSessionStatus` - For each configured BGP peer, this question will return the `Established_Status`. This is the status of the session after Batfish has completed analyzing the configurations and building the routing and forwarding tables. Any BGP peer which has `Established_Status` other than `ESTABLISHED` indicates a misconfiguration. 

In addition to these 4 questions, we have created questions designed to test routing policies, which we will cover in a subsequent notebook.

### View BGP Configuration for ALL devices
Batfish makes BGP Configuration settings in the network easily accessible. Let's take a look at how you can retrieve the specific information you want. Let's start with configuration attributes of the BGP process on all devices running BGP by using the question `bfq.bgpProcessConfiguration`.

In [4]:
# Get BGP process configuration information for ALL devices
bgp_config = bfq.bgpProcessConfiguration().answer().frame()
bgp_config

,Node,VRF,Router_ID,Confederation_ID,Confederation_Members,Multipath_EBGP,Multipath_IBGP,Multipath_Match_Mode,Neighbors,Route_Reflector,Tie_Breaker
0,e8,default,172.16.9.0,None,None,False,False,PATH_LENGTH,"['172.16.21.0/32', '172.16.21.1/32', '172.16.21.2/32']",False,ARRIVAL_ORDER
1,a6,default,172.16.21.0,None,None,False,False,PATH_LENGTH,"['172.16.7.0/32', '172.16.8.0/32', '172.16.9.0/32', '172.16.25.0/32', '172.16.25.1/32', '172.16.25.2/32']",False,ARRIVAL_ORDER
2,e6,default,172.16.7.0,None,None,False,False,PATH_LENGTH,"['172.16.21.0/32', '172.16.21.1/32', '172.16.21.2/32']",False,ARRIVAL_ORDER
3,e7,default,172.16.8.0,None,None,False,False,PATH_LENGTH,"['172.16.21.0/32', '172.16.21.1/32', '172.16.21.2/32']",False,ARRIVAL_ORDER
4,a16,default,172.16.24.1,None,None,False,False,PATH_LENGTH,"['172.16.16.0/32', '172.16.17.0/32', '172.16.18.0/32', '172.16.25.3/32', '172.16.25.4/32', '172.16.25.5/32']",False,ARRIVAL_ORDER
5,e3,default,172.16.4.0,None,None,False,False,PATH_LENGTH,"['172.16.20.0/32', '172.16.20.1/32', '172.16.20.2/32']",False,ARRIVAL_ORDER
6,a9,default,172.16.22.0,None,None,False,False,PATH_LENGTH,"['172.16.10.0/32', '172.16.11.0/32', '172.16.12.0/32', '172.16.25.0/32', '172.16.25.1/32', '172.16.25.2/32']",False,ARRIVAL_ORDER
7,c8,default,172.16.25.8,None,None,False,False,PATH_LENGTH,"['172.16.19.2/32', '172.16.20.2/32', '172.16.21.2/32', '172.16.22.2/32', '172.16.23.2/32', '172.16.24.2/32']",False,ARRIVAL_ORDER
8,c6,default,172.16.25.6,None,None,False,False,PATH_LENGTH,"['172.16.19.2/32', '172.16.20.2/32', '172.16.21.2/32', '172.16.22.2/32', '172.16.23.2/32', '172.16.24.2/32']",False,ARRIVAL_ORDER
9,a14,default,172.16.23.2,None,None,False,False,PATH_LENGTH,"['172.16.13.0/32', '172.16.14.0/32', '172.16.15.0/32', '172.16.25.6/32', '172.16.25.7/32', '172.16.25.8/32']",False,ARRIVAL_ORDER


Now let's drill into the configuration of a specific BGP session. Let's look at the sessions on `as2dept1`. To do this, we will use the `bfq.bgpPeerConfiguration` question. 

In [5]:
# Get all of the BGP peer configuration for as2dept1 devices
bgp_peer_config = bfq.bgpPeerConfiguration().answer().frame()
bgp_peer_config

,Node,VRF,Local_AS,Local_IP,Local_Interface,Confederation,Remote_AS,Remote_IP,Description,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
0,c5,default,25,172.16.25.5,None,None,20,172.16.20.1,None,False,None,None,[],[],False,False
1,c6,default,25,172.16.25.6,None,None,20,172.16.20.2,None,False,None,None,[],[],False,False
2,a12,default,23,172.16.23.0,None,None,25,172.16.25.2,None,False,None,None,[],[],False,False
3,c6,default,25,172.16.25.6,None,None,21,172.16.21.2,None,False,None,None,[],[],False,False
4,a0,default,19,172.16.19.0,None,None,2,172.16.2.0,None,False,None,None,[],[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,a14,default,23,172.16.23.2,None,None,25,172.16.25.8,None,False,None,None,[],[],False,False
212,a11,default,22,172.16.22.2,None,None,10,172.16.10.0,None,False,None,None,[],[],False,False
213,a14,default,23,172.16.23.2,None,None,25,172.16.25.7,None,False,None,None,[],[],False,False
214,a14,default,23,172.16.23.2,None,None,25,172.16.25.6,None,False,None,None,[],[],False,False


### View BGP Session Status

Now that we have seen the configuration for each peer on `as2dept1`, let's ensure that their configuration is compatible with their peers

The `bgpSessionCompatibility` question allows you to ensure that BGP sessions are compatibly configured, so that if there is IP reachability between the peers the sessions will be established. Compatiblity checks that the remote-as matches up on both ends, the correct update source is specified, peer-ip addresses match-up, etc...

In [6]:
# Check if the bgp Sessions on as2dept1 are properly configured
bgpSessCompat = bfq.bgpSessionCompatibility().answer().frame()
bgpSessCompat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Configured_Status
0,a0,default,19,None,172.16.19.0,1,e0,None,172.16.1.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
1,a0,default,19,None,172.16.19.0,2,e1,None,172.16.2.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
2,a0,default,19,None,172.16.19.0,3,e2,None,172.16.3.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
3,a0,default,19,None,172.16.19.0,25,c0,None,172.16.25.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
4,a0,default,19,None,172.16.19.0,25,c1,None,172.16.25.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
...,...,...,...,...,...,...,...,...,...,...,...,...
211,e8,default,9,None,172.16.9.0,21,a7,None,172.16.21.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
212,e8,default,9,None,172.16.9.0,21,a8,None,172.16.21.2,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
213,e9,default,10,None,172.16.10.0,22,a9,None,172.16.22.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH
214,e9,default,10,None,172.16.10.0,22,a10,None,172.16.22.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,UNIQUE_MATCH


Both of the configured BGP peers on `as2dept1` are compatible. We know that since the `Configured_Status` is `UNIQUE_MATCH`. So now let's check if they are established.

In [7]:
# Check if the bgp Sessions on as2dept1 are ESTABLISHED
bgpSessStat = bfq.bgpSessionStatus().answer().frame()
bgpSessStat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Established_Status
0,a0,default,19,None,172.16.19.0,1,e0,None,172.16.1.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
1,a0,default,19,None,172.16.19.0,2,e1,None,172.16.2.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
2,a0,default,19,None,172.16.19.0,3,e2,None,172.16.3.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
3,a0,default,19,None,172.16.19.0,25,c0,None,172.16.25.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
4,a0,default,19,None,172.16.19.0,25,c1,None,172.16.25.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
...,...,...,...,...,...,...,...,...,...,...,...,...
211,e8,default,9,None,172.16.9.0,21,a7,None,172.16.21.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
212,e8,default,9,None,172.16.9.0,21,a8,None,172.16.21.2,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
213,e9,default,10,None,172.16.10.0,22,a9,None,172.16.22.0,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED
214,e9,default,10,None,172.16.10.0,22,a10,None,172.16.22.1,['IPV4_UNICAST'],EBGP_SINGLEHOP,NOT_ESTABLISHED


Both sessions are established. Let's see if there are any configured BGP sessions, on any other device in the network, that are not established

In [8]:
# Find any BGP sessions in the network that are NOT ESTABLISHED
bgpSessStat = bfq.bgpSessionStatus().answer().frame()
bgpSessStat[bgpSessStat['Established_Status'] == 'ESTABLISHED']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Established_Status


Looking at the `Established_Status` column we see that there are a lot of sessions that are configured, but not established. Let's dig into these issues. 

First, let's find all of the sessions that are not compatibly configured. For that we are looking for sessions which are not either a `UNIQUE_MATCH` or `DYNAMIC_MATCH`. The latter is for `dynamic` peers (these are peers configured to use a `listen-range`).

### Debug NOT_COMPATIBLE sessions 

In [9]:
# Find BGP sessions that are not compatibly configured - i.e Batfish does not identify them as being a UNIQUE_MATCH or a DYNAMIC_MATCH
bgpSessCompat = bfq.bgpSessionCompatibility().answer().frame()
bgpSessCompat[~bgpSessCompat['Configured_Status'].isin(["UNIQUE_MATCH", "DYNAMIC_MATCH"])]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Configured_Status


We see 4 entries in this table even though we only saw 3 BGP session with thes status NOT_COMPATIBLE. This means that despite not having a `Configured_Status` of `UNIQUE_MATCH` or `DYNAMIC_MATCH` one of these sessions was indeed established. This typically occurs if the mis-configuration is such that the session can ONLY be established when initiated from one side, but not the other. The likely candidate in this output is the `as2border2` session to `2.1.2.2` that has status of `LOCAL_IP_UNKNOWN_STATICALLY`.

### Debug UNKNOWN_REMOTE peer on as1border2

Batfish deems the BGP peer `10.14.22.4` is not compatible because it cannot find a device in the snapshot that has that IP address configured on any interface. That is why the status is `UNKNOWN_REMOTE`.
This will occur in  most networks, since you will not have the configurations of the devices for your external peers (ISPs, content partners, etc...). We can easily verify this by checking the output of `bfq.ipOwners`

In [10]:
# check if there is a node in the network that has the `10.14.22.4` on an interface
ipOwn = bfq.ipOwners().answer().frame()
ipOwn[ipOwn['IP']=='10.14.22.4']

,Node,VRF,Interface,IP,Mask,Active


### Debug LOCAL_IP_UNKNOWN_STATICALLY on as2border2

Now, let's dig into the sessions that are `LOCAL_IP_UNKNOWN_STATICALLY`.

An iBGP session will have the status `LOCAL_IP_UNKNOWN_STATICALLY` if you are missing the `update-source` command.

So, in this case, it is likely that the issue is that `as2border2` is missing the `update-source` command for the BGP session. This is needed for iBGP sessions to ensure the peers pick the correct IP address to use when trying to establish the TCP session. We can find out the target remote node by looking at the `bfq.ipOwners` output

In [11]:
#find the device(s) that own these ip addresses
ipOwn[ipOwn['IP'].isin(['2.1.2.2'])]

,Node,VRF,Interface,IP,Mask,Active


So this session is supposed to be between `as2border2` and `as2core2`. Let's see if this session is established.

In [12]:
# check if either direction of the session can be established
bgpSessStat[((bgpSessStat['Local_IP']=='2.1.1.2') & (bgpSessStat['Remote_IP']=='2.1.2.2')) | ((bgpSessStat['Local_IP']=='2.1.2.2') & (bgpSessStat['Remote_IP']=='2.1.1.2'))]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Established_Status


This confirms that our theory. We are missing the `update-source` command on `as2border2` which prevents it from initiating the BGP session. But since `as2core2` is properly configured, the session is established. 

### Debug NO_LOCAL_IP on as1border1

In [13]:
# Identify the BGP sessions on as1border1 that are not compatible with Configured_Status of NO_LOCAL_IP
bgpSessCompat[(bgpSessCompat['Configured_Status'] == 'NO_LOCAL_IP') & (bgpSessCompat['Node']=='as1border1')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Configured_Status


For an `EBGP_SINGLEHOP` session to have `CONFIGURED_STATUS` of `NO_LOCAL_IP` this typically means that no interface exists on the box which is in the same subnet as the BGP peer's IP address. This can easily be checked by looking at the IP addresses configued on `as1border1`.

In [14]:
# Identify the IP addresses that are owned by as1border1
ipOwn[ipOwn['Node']=='as1border1']

,Node,VRF,Interface,IP,Mask,Active


As you can see there are no interfaces with addresses that would be in the same subnet as `3.2.2.2` and `5.6.7.8`. There could be many explanations for this:

1) BGP peers were configured before the physical connection to neighboring routers was up.

2) The user simply configured the wrong BGP peer address.

3) The interfaces used to exist but were decommissioned, but the BGP config was not cleaned up at the same time.

4) The session is meant to be be an eBGP multi-hop session, but the user didn't add the `ebgp multihop` configuration option and specify an `update-source`.

Batfish determines the Local_IP for each BGP session, either based on explicit configuration with `updates-source foo` (or equivalent non-IOS command) for iBGP sessions or eBGP multi-hop sessions,
or by determining the interface the router will use to send packets towards the BGP peer. The latter method requires the route to the peer to be known.

So, if there is no route to the configured peer, or the configured peer does not exist in the snapshot, you will see this status. We can check the output of `bfq.routes` and `bfq.ipOwners` questions to dig into this

In [15]:
# Find owner of IP addresses for the incompatible BGP sessions on as1border`
bad_bgp_peer = ['3.2.2.2', '5.6.7.8']
ipOwn[ipOwn['IP'].isin(bad_bgp_peer)]

,Node,VRF,Interface,IP,Mask,Active


So we can see that `5.6.7.8` does not exist in the network. This either means there is a mis-configuration, or the device is just not expected to be in the snapshot.

Now let's dig into the peer `3.2.2.2`, which we know is the Loopback interface on `as3border2`

In [16]:
# retrieve the routing table entry for as3border2 loopback0 - 3.2.2.2/32 on as1border1
routes = bfq.routes(network='3.2.2.2/32').answer().frame()
routes[routes['Node']=='as1border1']

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


The specific /32 is not present on `as1border1`. What about other routers?

In [17]:
# retrieve the routing table entry for as3border2 loopback0 - 3.2.2.2/32 on ALL routers in the network
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


We can see that this route does not leave `as3`, which is why `as1border1` is unable to established the configured BGP session.
This session should have been configured as an eBGP multi-hop session with static routes pointing to the appropriate interface and next-hop

### Debugging BGP sessions that are NOT_ESTABLISHED

We have root-caused the `NOT_COMPATIBLE` sessions, now let's dig into the ones that are `NOT_ESTABLISHED`.

In [18]:
# Find all BGP sessions in the network that were compatible but NOT_ESTABLISHED
bgpSessStat[bgpSessStat['Established_Status'] == 'ESTABLISHED']

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_Interface,Remote_IP,Address_Families,Session_Type,Established_Status


The  reasons for a session that is compatiable (`UNIQUE_MATCH` or `DYNAMIC_MATCH`) to not get established would be 1) missing routes or 2) some ACL in the path blocking traffic. Let's check the routing tables on `as2core1`

In [19]:
# retrieve routing table for as2core1 and check the route to the BGP peer 2.1.1.1 - as2border1
routes = bfq.routes(nodes='a0').answer().frame()
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,a0,default,10.0.3.2/32,None,AUTO/NONE(-1l),Serial1,connected,0,0,None
1,a0,default,10.0.55.1/32,None,AUTO/NONE(-1l),Serial4,connected,0,0,None
2,a0,default,10.0.54.1/32,None,AUTO/NONE(-1l),Serial3,connected,0,0,None
3,a0,default,10.0.6.2/32,None,AUTO/NONE(-1l),Serial2,connected,0,0,None
4,a0,default,10.0.0.2/32,None,AUTO/NONE(-1l),Serial0,connected,0,0,None
5,a0,default,10.0.56.1/32,None,AUTO/NONE(-1l),Serial5,connected,0,0,None
6,a0,default,172.16.19.0/32,None,AUTO/NONE(-1l),Loopback0,connected,0,0,None


In [20]:
# retrieve routing table for as2core1 and check the route to the BGP peer 2.1.1.1 - as2border1
routes = bfq.routes(nodes='aggregation-2').answer().frame()
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


As we can see `as2core1` has a route to the configured neighbor `2.1.1.1`. What about the other routers?

In [21]:
# retrieve routing table for as2border1 to check if it has a route to as2core1 loopback0 - 2.1.2.1/32
routes=bfq.routes(nodes='core-0').answer().frame()
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


`as2border1` does not have a route to the loopback address `2.1.2.1` of `as2core1`. Let's also check `as2border2`.

In [22]:
# retrieve routing table for as2border2 to check if it has a route to as2core1 loopback0 - 2.1.2.1/32
routes=bfq.routes(nodes='aggregation-1').answer().frame()
routes

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag


Neither border router has a route to the loopback of `as2core1`. Since the loopback is supposed to be distributed via OSPF, next step is to look at the OSPF configuration on `as2core1`

From the snippet of the configuration of `as2core1`, we can see that the Loopback address isn't part of the OSPF process:
```
interface Loopback0
 ip address 2.1.2.1 255.255.255.255

router ospf 1
 router-id 2.1.2.1
 !network 2.0.0.0 0.255.255.255 area 1
 network 2.12.0.0 0.0.255.255 area 1
 network 2.23.0.0 0.0.255.255 area 1
 ```
 
 This explains why the BGP session wasn't established.


With that we have root-caused all of the BGP sessions that were `NOT_ESTABLISHED` or `NOT_COMPATIBLE`. 

### Summary


Batfish allows you to easily retrieve information about BGP configuration of all devices and peers, as well as status of each peer. With Batfish you can ensure that no change is pushed to the network that would cause a BGP session to not come up.

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics like validating routing policy. Stay tuned! 

Want to learn more? Come find us on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZT     U2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [GitHub](https://github.com/batfish/batfish)